In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [2]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, names
import datetime
from scipy.stats import ttest_ind 
import csv

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
## add after creating spark_PCA.py
sc = SparkContext(master="local[3]") 
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
from pyspark.sql import Row
from pyspark.sql.types import *

In [5]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [6]:
park_data = pd.read_json('Datasources/other general/parks_datasd.geojson', orient = 'records')

In [7]:
df_geo = pd.DataFrame()
count = 0

for idx in park_data.index:
    k = pd.DataFrame.from_dict({count: {'center_x': park_data['features'][idx]['properties']['center_x'], 
                                            'center_y': park_data['features'][idx]['properties']['center_y'],
                                             }}, orient = 'index')
    df_geo = df_geo.append(k)
    
    count = count + 1

In [8]:
schema = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)])

In [9]:
schema_two = StructType([StructField("park_id", IntegerType(), True),
                    StructField("park_longitude", DoubleType(), True),
                    StructField("park_latitude", DoubleType(), True)])

In [10]:
spql_listings_info = sqlContext.createDataFrame(listings[['id', 'longitude', 'latitude']], schema)

In [11]:
df_geo['park_id'] = range(len(df_geo))

In [12]:
spql_parks_info = sqlContext.createDataFrame(df_geo[['park_id', 'center_x', 'center_y']], schema_two)

In [13]:
spql_listings_info_rdd = spql_listings_info.rdd.map(lambda row:(row.id, row.longitude,row.latitude))

In [14]:
spql_parks_info_rdd = spql_parks_info.rdd.map(lambda row:(row.park_id, row.park_longitude,row.park_latitude))

In [15]:
spql_listings_and_parks_info_rdd = spql_listings_info_rdd.cartesian(spql_parks_info_rdd)

In [16]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_rdd.map(lambda x: (x[0][0], x[0][1], x[0][2], x[1][0], x[1][1], x[1][2]))

In [17]:
def distance_calculation(x):
        p_one = float(x[2])
        q_one = float(x[1])
        p_two = float(x[5])
        q_two = float(x[4])
        lon_diff = (q_one - q_two)*np.pi/180
        lat_diff = (p_one - p_two)*np.pi/180
        a = np.sin(lat_diff/2)**2 + np.cos(p_one*np.pi/180)*np.cos(p_two*np.pi/180)*(np.sin(lon_diff/2)**2)
        c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        d = 6371.00*float(c)
        return tuple(list(x) + [d])

In [18]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_rdd.map(distance_calculation)

In [19]:
spql_listings_and_parks_info_rdd.take(2)

[(11204286,
  -117.00194036983295,
  32.581881678016465,
  0,
  -117.113,
  32.708,
  8.729150422837035),
 (11204286,
  -117.00194036983295,
  32.581881678016465,
  1,
  -117.161,
  32.708,
  10.228095266922994)]

In [20]:
schema_three = StructType([StructField("id", IntegerType(), True),
                    StructField("longitude", DoubleType(), True),
                    StructField("latitude", DoubleType(), True)] + [StructField("park_id", IntegerType(), True),
                    StructField("park_longitude", DoubleType(), True),
                    StructField("park_latitude", DoubleType(), True), StructField("distance", DoubleType(), True)])

In [21]:
spql_listings_and_parks_info_df = sqlContext.createDataFrame(spql_listings_and_parks_info_rdd, schema_three)

In [22]:
spql_listings_and_parks_info_rdd = spql_listings_and_parks_info_df.rdd.map(lambda row: (row.id, row.longitude, row.latitude, row.park_id, row.park_longitude, row.park_latitude, row.distance))

In [23]:
close_parks_count = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 16).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [24]:
closest_park_distance = spql_listings_and_parks_info_rdd.map(lambda x: (x[0], x[-1])).reduceByKey(lambda x, y : min(x, y))

In [25]:
park_distance  = closest_park_distance.map(lambda x: x[1])

In [26]:
park_distance.take(5)

[0.4226653619251699,
 0.22449796494527174,
 0.40689894777771984,
 0.25248363187723993,
 0.2598624210631214]

In [27]:
close_parks_count_one = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 1).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_parks_count_three = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 3).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_parks_count_five = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 5).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)
close_parks_count_ten = spql_listings_and_parks_info_rdd.filter(lambda x: x[-1] <= 10).map(lambda x: (x[0], 1)).reduceByKey(lambda x, y : x + y)

In [28]:
close_parks_count_three_value = close_parks_count_three.map(lambda x: x[1])
close_parks_count_five_value = close_parks_count_five.map(lambda x: x[1])
close_parks_count_ten_value = close_parks_count_ten.map(lambda x: x[1])
close_parks_count_value = close_parks_count.map(lambda x: x[1])

In [29]:
count = 1
Writer = pd.ExcelWriter('listings_parks_information_two.xlsx')

for w in [close_parks_count_one, close_parks_count_three, close_parks_count_five, close_parks_count_ten, close_parks_count, closest_park_distance]:
    listings_parks_information = w.toDF().toPandas()
    listings_parks_information.to_excel(Writer, 'Sheet' + str(count))
    count = count + 1
    
Writer.save()